In [68]:
import pandas
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

import time
import datetime

In [55]:
features = pandas.read_csv('./features.csv', index_col='match_id')
features.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52.0,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5.0,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13.0,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27.0,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16.0,2449,0,4,1974,3,63


In [56]:
X = features.loc[:, 'lobby_type':'dire_first_ward_time']
y = features.radiant_win
X.fillna(1000000000, inplace=True)

In [57]:
X.head()

,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_hero,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
0,7,11,5,2098,1489,20,0,0,7,67,...,0,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0
1,0,42,4,1188,1033,9,0,1,12,49,...,0,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0
2,7,33,4,1319,1270,22,0,0,12,98,...,1,-39.0,45.0,-77.0,221.0,3,4,3,1,13.0
3,1,29,4,1779,1056,14,0,0,5,30,...,0,-30.0,124.0,-80.0,184.0,0,4,2,0,27.0
4,7,13,4,1431,1090,8,1,0,8,27,...,0,46.0,182.0,-80.0,225.0,6,3,3,0,-16.0


In [58]:
X.fillna(1000000, inplace=True)

In [70]:
cv = KFold(n_splits=5, shuffle=True, random_state=241)
for e in [10, 20, 30, 40, 50, 60, 70, 80]:
    scores = []
    
    start_time = datetime.datetime.now()
    
    for train_index, test_index in cv.split(all_data):
        X_train = X.iloc[train_index]
        y_train = y.iloc[train_index]

        X_test = X.iloc[test_index]
        y_test = y.iloc[test_index]

        model = GradientBoostingClassifier(n_estimators=e, random_state=241)
        model.fit(X_train, y_train)

        predicts = model.predict_proba(X_test)[:, 1]
        scores.append(roc_auc_score(y_true=y_test, y_score=predicts))
        
    print('Time elapsed: ', datetime.datetime.now() - start_time)
    print(e, ': ', round(np.mean(scores), 2))
    print('')

Time elapsed:  0:00:18.071140
10 :  0.67

Time elapsed:  0:00:33.244423
20 :  0.68

Time elapsed:  0:00:46.239129
30 :  0.69

Time elapsed:  0:01:01.692610
40 :  0.69

Time elapsed:  0:01:16.111377
50 :  0.7

Time elapsed:  0:01:31.572111
60 :  0.7

Time elapsed:  0:01:46.349095
70 :  0.7

Time elapsed:  0:02:03.656305
80 :  0.7

